# 0.1 Import Packages

In [2]:
import pandas as pd
import datetime as dt
import numpy as np
from pymongo import MongoClient
from pprint import pprint
import re
from langdetect import detect  #detects what language is written
from  tqdm import tqdm #timing package
from cleantext import clean
import dnspython

In [3]:
import sys
sys.path.append("D:\Metis\Projects\Project 4\GetOldTweets-python")

In [4]:
import got3
import time

# 0.2 Connecting to online mongoDB

In [11]:
sdate = pd.to_datetime("2020-01-01").date()
edate = pd.to_datetime("2020-05-01").date()
date_list = pd.date_range(sdate,edate-dt.timedelta(days=1),freq='d')
# str(date_list[0].date())

In [14]:
client = MongoClient("mongodb+srv://<username>:<password>@covid19-mask-scacy.mongodb.net/test?retryWrites=true&w=majority")
db = client.test

In [ ]:
db.create_collection('tweets_1/1_to_5/1')
t1151 = client.mask['tweets_1/1_to_5/1']

In [17]:
t59 = client.mask['twitter_5/9']

In [18]:
len(list(t59.find()))

28986

# 0.3 Getting old tweets (1/1 to 5/1, 2020)

In [ ]:
for d in range(0, len(date_list)):
    tweetCriteria = got3.manager.TweetCriteria().setQuerySearch('mask').setSince(str(date_list[d].date())).setUntil(str(date_list[d+1].date())).setMaxTweets(200).setLang('en')
    for tweet in got3.manager.TweetManager.getTweets(tweetCriteria):
        try:
            print (tweet.text)
            print (tweet.date.date())
            t1151.insert_one({'tweet': tweet.text, 'date':str(date_list[d].date())})
        except:
            print ('something is wrong')

    
    
#     since the rate of acquisition is extremely fast (200 a pop), adding a sleep timer to make sure data are inserted correctly
    time.sleep(5)

In [ ]:
len(list(t1151.find()))

In [ ]:
df = pd.DataFrame(list(t1151.find()))


In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.date.unique()

In [23]:
import os
def df_to_csv(df, outname = 'name.csv', outdir = './CSVs'):
    
    if not os.path.exists(outdir):
        os.mkdir(outdir)

    fullname = os.path.join(outdir, outname)    

    df.to_csv(fullname,index=False)

### 0.3.1 Raw data is collected, see "Historical Data Processing" notebook for processing details

In [ ]:
df_to_csv(df, '4_1_5_1_final.csv')

# 1.0 Building specialized Word2Vec Model
##### Based on tutorial here: https://www.kaggle.com/pierremegret/gensim-word2vec-tutorial
##### Note that I didn't end up using the model that I built, mainly because the model size is too large (45 mb) for my app on the free server

In [7]:
from gensim.models.phrases import Phrases, Phraser
from nltk import bigrams
from collections import defaultdict
import multiprocessing
from gensim.models import Word2Vec

In [ ]:
sent = [row.split() for row in df['tweet']]

In [ ]:
phrases = Phrases(sent, min_count=5, progress_per=10000)

In [ ]:
bigram = Phraser(phrases)
sentences = bigram[sent]


In [ ]:
word_freq = defaultdict(int)
for sent in sentences:
    for i in sent:
        word_freq[i] += 1
len(word_freq)

In [ ]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

In [ ]:
cores = multiprocessing.cpu_count()
cores

In [ ]:
w2v_model = Word2Vec(min_count=2,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=10,
                     workers=cores-1)

In [ ]:
w2v_model.build_vocab(sentences, progress_per=10000)

In [ ]:
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)

In [ ]:
w2v_model.init_sims(replace=True)

In [1]:
w2v_model.wv.most_similar(positive=["mask"])

NameError: name 'w2v_model' is not defined

In [12]:
similar = word_vectors.most_similar(positive=["face_mask"])

In [17]:
len(similar)

10

In [15]:
similar[0][0]

'at'

In [ ]:
for term,count in similar:
    related_words[term] = [df['sentiment'].mean(), count]

In [ ]:
w2v_model.wv.most_similar(positive=["infection"])

In [ ]:
w2v_model.wv.similarity("death", 'mask')

In [ ]:
w2v_model.wv.doesnt_match(['mask', 'hand_sanitizer', 'toilet_paper'])

In [6]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
 
import seaborn as sns
sns.set_style("darkgrid")

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import random as random

In [ ]:
rand_words = []
for i in range (0, 11):
    rand_words.append(sent[random.randint(0, len(sent)-1)])
rand_words

In [ ]:
def tsnescatterplot(model, word, list_names):
    """ Plot in seaborn the results from the t-SNE dimensionality reduction algorithm of the vectors of a query word,
    its list of most similar words, and a list of words.
    """
    arrays = np.empty((0, 300), dtype='f')
    word_labels = [word]
    color_list  = ['red']

    # adds the vector of the query word
    arrays = np.append(arrays, model.wv.__getitem__([word]), axis=0)
    
    # gets list of most similar words
    close_words = model.wv.most_similar([word])
    
    # adds the vector for each of the closest words to the array
    for wrd_score in close_words:
        wrd_vector = model.wv.__getitem__([wrd_score[0]])
        word_labels.append(wrd_score[0])
        color_list.append('blue')
        arrays = np.append(arrays, wrd_vector, axis=0)
    
    # adds the vector for each of the words from list_names to the array
    for wrd in list_names:
        wrd_vector = model.wv.__getitem__([wrd])
        word_labels.append(wrd)
        color_list.append('green')
        arrays = np.append(arrays, wrd_vector, axis=0)
        
    # Reduces the dimensionality from 300 to 50 dimensions with PCA
    reduc = PCA(n_components=15,svd_solver='full').fit_transform(arrays)
    
    # Finds t-SNE coordinates for 2 dimensions
    np.set_printoptions(suppress=True)
    
    Y = TSNE(n_components=2, random_state=0, perplexity=15).fit_transform(reduc)
    
    # Sets everything up to plot
    df = pd.DataFrame({'x': [x for x in Y[:, 0]],
                       'y': [y for y in Y[:, 1]],
                       'words': word_labels,
                       'color': color_list})
    
    fig, _ = plt.subplots()
    fig.set_size_inches(9, 9)
    
    # Basic plot
    p1 = sns.regplot(data=df,
                     x="x",
                     y="y",
                     fit_reg=False,
                     marker="o",
                     scatter_kws={'s': 40,
                                  'facecolors': df['color']
                                 }
                    )
    
    # Adds annotations one by one with a loop
    for line in range(0, df.shape[0]):
         p1.text(df["x"][line],
                 df['y'][line],
                 '  ' + df["words"][line].title(),
                 horizontalalignment='left',
                 verticalalignment='bottom', size='medium',
                 color=df['color'][line],
                 weight='normal'
                ).set_size(15)

    
    plt.xlim(Y[:, 0].min()-50, Y[:, 0].max()+50)
    plt.ylim(Y[:, 1].min()-50, Y[:, 1].max()+50)
            
    plt.title('t-SNE visualization for {}'.format(word.title()))

In [ ]:
tsnescatterplot(w2v_model, "mask", [t[0] for t in w2v_model.wv.most_similar(positive=["mask"], topn=20)][10:])

In [ ]:
tsnescatterplot(w2v_model, 'good', [i[0] for i in w2v_model.wv.most_similar(negative=["good"])])


In [ ]:
w2v_model.save("word2vec.model")

In [ ]:
from sklearn.cluster import KMeans

In [9]:
word_vectors = Word2Vec.load("./models/word2vec.model").wv

In [ ]:
model = KMeans(n_clusters=2, max_iter=1000, random_state=True, n_init=50).fit(X=word_vectors.vectors)
positive_cluster_center = model.cluster_centers_[0]
negative_cluster_center = model.cluster_centers_[1]

In [ ]:
model.cluster_centers_

In [ ]:
word_vectors.similar_by_vector(model.cluster_centers_[0], topn=10, restrict_vocab=None)

In [ ]:
words = pd.DataFrame(word_vectors.vocab.keys())
words.columns = ['words']
words['vectors'] = words.words.apply(lambda x: word_vectors.wv[f'{x}'])
words['cluster'] = words.vectors.apply(lambda x: model.predict([np.array(x)]))
words.cluster = words.cluster.apply(lambda x: x[0])
words['cluster_value'] = [1 if i==0 else -1 for i in words.cluster]
words['closeness_score'] = words.apply(lambda x: 1/(model.transform([x.vectors]).min()), axis=1)
words['sentiment_coeff'] = words.closeness_score * words.cluster_value

In [ ]:
words.head()

In [ ]:
words[['words', 'sentiment_coeff']].to_csv('sentiment_dictionary.csv', index=False)

In [ ]:
df.head()

In [ ]:
# file_cleaned = df.dropna().drop_duplicates().reset_index(drop=True)
# file_cleaned.rate.value_counts()/len(file_cleaned)


In [ ]:
file_export = df.copy()
file_export.tweet = file_export.tweet.apply(lambda x: ' '.join(bigram[x]))

In [ ]:
file_export[['tweet', 'date']].to_csv('cleaned_dataset.csv', index=False)